### Pytorch
파이토치를 좋아하는 이유는 간단하다.     
코드를 보기 편하기 때문.    
모든 예제는      
[링크](http://bit.ly/2U7ttYT)에서 보면서 했다.    


In [1]:
import torch

print(torch.__version__)

1.13.1


### 선형 회귀 분석
선형회귀분석은 간단히 주어진 데이터를 가장 잘 설명하는 직선 하나이다.    
하나의 독립변수에는 단순선형회귀라고 하고, 여러개의 독립변수인 경우에는 다중선형회귀라고 한다.     


In [2]:
#맥북 기준이라서 cuda는 설치되어있지 않다.
#파이토치에서는 데이터의 기본 단위로 텐서를 사용한다. 
#텐서는 다차원 배열이라고 생각하면 된다. 
X = torch.randn(2,3)
print(X)

tensor([[ 0.3578,  0.1575, -1.4022],
        [-0.4805,  0.2136,  0.8910]])


rand을 사용해서 2*3행렬을 만들었다. 각 값은 랜덤 값이 들어가게 된다.    
또한 다음과 같이 설정할 수 있다.

In [4]:
X1 = torch.tensor([[1,2,3],[4,5,6]])
print(X1)

tensor([[1, 2, 3],
        [4, 5, 6]])


In [5]:
#이렇게 초기화된 텐서는 인수로 data, dtype, device, requires_grad를 받는다.
#data는 어떤 데이터가 들어갈지, dtype은 어떤 데이터 타입이 들어갈지, device는 cpu에서 할지 gpu에서 할지 requires_grad는 텐서에 대한 기울기를 저장할지 여부이다.
#float 는 32비트 부동소수점, double은 64비트 부동소수점이다. 보통 float를 사용한다. 
#requires_grad는 기본은 False이지만, 변수의 기울기를 저장하려면 True를 줘서 저장할 수 있다.
x_tensor = torch.tensor(data=[2.0,3.0],requires_grad=True)
print(x_tensor)

tensor([2., 3.], requires_grad=True)


In [6]:
#이렇게 생성한 텐서를 가지고 연산 그래프를 생성하면 연산 그래프는 어떠한 결과를 산출한다. 
#결과값과 목표값의 차이가 곧 오차이고, 경사하강법을 지정한 횟수만큼 반복하면 변수 w,b가 오차를 감소시키는 방향으로 변하는 것을 확인할 수 있다.
y = x_tensor**2
z = 2*y +3

target = torch.tensor([3.0,4.0])
loss = torch.sum(torch.abs(z-target))
loss.backward()

print(x_tensor.grad,y.grad,z.grad)

tensor([ 8., 12.]) None None


/Users/baghyeonseog/opt/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/build/aten/src/ATen/core/TensorBody.h:485.)
  # Remove the CWD from sys.path while we load stuff.


In [7]:
#이번엔 선형회귀분석 모델을 만들어서 기울기를 계산하고 w,b를 업데이트하는 전체 과정을 보여준다.
import torch.nn as nn #torch.nn에는 다양한 신경망 모델이 포함되어있다.
import torch.optim as optim #optim에는 다양한 경사하강법 알고리즘이 있다.
import torch.nn.init as init #텐서의 초기값을 설정해줄 수 있다.

num_data = 1000
num_epoch = 50

x = init.uniform_(torch.Tensor(num_data,1),-10,10)#num_data*1모양의 텐서를 생성하고, -10부터 10까지 균등하게 초기화함
noise = init.normal_(torch.FloatTensor(num_data,1),std=1)#노이즈는 표준정규분포를 따르는 가우시안 노이즈 사용
y = 2*x + 3
y_noise = y+noise #현실세계 데이터는 노이즈가 있을 수 있기 때문에 노이즈를 더해줌



In [8]:
model = nn.Linear(1,1) #input size output size 단순 선형회귀이니 하나의 값이 들어가면 하나의 output
loss_func = nn.L1Loss() #맨허튼 거리 사용하는 loss사용
optimizer = optim.SGD(model.parameters(),lr=0.01)#경사하강법 초기화 SGD로, 학습률 0.01 모델의 파라미터 최적화를 위해서 model.parameters를 인자로 넘겨줌

label = y_noise

for i in range(num_epoch):
    optimizer.zero_grad()#지난번에 계산했던 기울기 0으로 초기화
    output = model(x)
    loss = loss_func(output,label)
    loss.backward()#w와 b에 대한 기울기 계산
    optimizer.step()#기울기에 학습률 0.01을 곱해줌
    if i % 10 == 0:
        print(loss)

tensor(9.4881, grad_fn=<MeanBackward0>)
tensor(7.1967, grad_fn=<MeanBackward0>)
tensor(5.0716, grad_fn=<MeanBackward0>)
tensor(3.5250, grad_fn=<MeanBackward0>)
tensor(3.0331, grad_fn=<MeanBackward0>)
